In [ ]:
from PIL import Image

# Set page config
st.set_page_config(page_title="Finance Agent", layout="centered")

# Custom CSS to set a max-width for the content
st.markdown("""
    <style>
    .reportview-container .main .block-container {
        max-width: 1000px;
        padding-top: 2rem;
        padding-bottom: 2rem;
        margin: 0 auto;
    }
    </style>
""", unsafe_allow_html=True)

# Rest of the code remains the same as before
@contextmanager
def st_redirect(src, dst):
    placeholder = st.empty()
    output_func = getattr(placeholder, dst)

    with StringIO() as buffer:
        old_write = src.write

        def new_write(b):
            if add_script_run_ctx():
                buffer.write(b)
                output_func(buffer.getvalue())
            else:
                old_write(b)

        try:
            src.write = new_write
            yield
        finally:
            src.write = old_write

@contextmanager
def st_stdout(dst):
    with st_redirect(sys.stdout, dst):
        yield

@contextmanager
def st_stderr(dst):
    with st_redirect(sys.stderr, dst):
        yield

st.title("Finance Agent Query Interface")

# Create a container for the main content
main_container = st.container()

with main_container:
    # Create two columns: left for main content, right for logs
    left_column, right_column = st.columns([2, 1])

    # Left column: Design diagram and query input
    with left_column:
        # Display design diagram
        st.subheader("Design Diagram")
        image = Image.open("assets/design_diagram.png")
        st.image(image, use_column_width=True)

        # Query input
        st.subheader("Query Input")
        query = st.text_area("Enter your finance-related query:", height=100)
        run_button = st.button("Run Query", type="primary")

    # Right column: Process Log
    with right_column:
        st.subheader("Process Log")
        log_container = st.empty()

    # Main content area for results (below query in left column)
    result_container = left_column.container()

    if run_button and query:
        with log_container:
            with st_stdout("code"):
                result = asyncio.run(run_finance_agent(query))

        with result_container:
            st.success("Query processing completed!")
            
            # Display graph database results if available
            if 'graph_db_result' in result:
                st.subheader("Graph Database Results")
                st.json(result['graph_db_result'])
            
            # Display expanded queries if available
            if 'expanded_queries' in result:
                st.subheader("Expanded Queries")
                for i, eq in enumerate(result['expanded_queries'], 1):
                    st.write(f"{i}. {eq}")
            
            # Display subgraph results if available
            if 'subgraph_results' in result:
                st.subheader("Subgraph Results")
                for subgraph, subresult in result['subgraph_results'].items():
                    with st.expander(f"Results from {subgraph}"):
                        st.json(subresult)
            
            # Display final response
            st.subheader("Final Response")
            st.write(result.get('final_response', 'No final response available.'))

    else:
        result_container.info("Enter a query and click 'Run Query' to start processing.")

# Footer
st.markdown("---")
st.markdown("© 2023 Finance Agent. All rights reserved.")